<a href="https://colab.research.google.com/github/honor200/honor-diss/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 111.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.9 MB/s eta 0:00:00


In [ ]:
pip install --upgrade transformers

In [ ]:
import pandas as pd
import string
import re
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

import tensorflow as tf

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = pd.read_csv("aft.csv")
df.drop(['provenance','notes','source','data_source','date_obtained'], axis = 1, inplace = True)
df.head()

,atu_id,tale_title,text
0,910B,The Highlander Takes Three Advices from the En...,In one of the glens of Cantire there lived a y...
1,910B,The Prince Who Acquired Wisdom,There was once a raja who had an only son and ...
2,910B,The Three Admonitions,A man once left his country to go to foreign p...
3,910B,The Three Advices,The stories current among the Irish peasantry ...
4,910B,The Three Advices Which the King with the Red ...,"The name of the young chief was Illan, called ..."


In [ ]:
# Removing text that is in a class of its own (necessary for oversampling)
class_counts = df['atu_id'].value_counts()
classes_with_few_instances = class_counts[class_counts < 10].index

# Create a boolean mask to filter rows with a class count less than 10
mask = df['atu_id'].isin(classes_with_few_instances)

# Remove rows with class count less than 10 from the dataframe
df = df[~mask]

len(df)

913

In [ ]:
df.head()

,atu_id,tale_title,text
5,1430,Buttermilk Jack,"Oh mother, my buttermilk I will sell, And all ..."
6,1430,Day-Dreaming,Now there was once a man at Bagdad who had sev...
7,1430,Lazy Heinz,"Heinz was lazy, and although he had nothing el..."
8,1430,Lean Lisa,Lean Lisa was not at all like Lazy Heinz and F...
9,1430,Sheik Chilli,The hero of this story was one day walking alo...


In [ ]:
df.shape

(913, 3)

In [ ]:
text = df.text.values
atu_id = df.atu_id.values

In [ ]:
print(text.shape)
print(atu_id.shape)

(913,)
(913,)


In [ ]:
train_input, val_input, train_label, val_label = train_test_split(text, atu_id)

In [ ]:
train_input.shape

(684,)

In [ ]:
train_label.shape

(684,)

In [ ]:
#from transformers import BertTokenizer, TFBertModel, BertConfig, TFBertForSequenceClassification

from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification

In [ ]:
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
print('Actual text ', text[0])

Actual text  Oh mother, my buttermilk I will sell, And all for a penny as you med zee; And with my penny then I will buy eggs, Vor I shall have seven for my penney. Oh mother, I'll set them all under our hen, And seven cock chickens might chance for to be; But seven cock chickens or seven cap hens, There'll be seven half-crownds for me. Oh, I'll go carry them to market, mother, And nothing but vine volk shall I zee; And with my money then I will buy land, Zo as a landlord I med be. 'Oh my dear zon, wilt thee know me, When thee hast gotten great store of wealth?' 'Oh, my dear mother, how shall I know thee, When I shall hardly know my own self?' Zo aal you as has got an old hen for to sett, Both by night and by day mind you has her well watched, Lest you should be like unto Buttermilk Jack, To reckon your chickens before thay are hatched.


In [ ]:
print('Tokens ', tokenizer.tokenize(text[0]))

Tokens  ['oh', 'mother', ',', 'my', 'butter', '##mi', '##lk', 'i', 'will', 'sell', ',', 'and', 'all', 'for', 'a', 'penny', 'as', 'you', 'med', 'zee', ';', 'and', 'with', 'my', 'penny', 'then', 'i', 'will', 'buy', 'eggs', ',', 'vo', '##r', 'i', 'shall', 'have', 'seven', 'for', 'my', 'penn', '##ey', '.', 'oh', 'mother', ',', 'i', "'", 'll', 'set', 'them', 'all', 'under', 'our', 'hen', ',', 'and', 'seven', 'cock', 'chickens', 'might', 'chance', 'for', 'to', 'be', ';', 'but', 'seven', 'cock', 'chickens', 'or', 'seven', 'cap', 'hen', '##s', ',', 'there', "'", 'll', 'be', 'seven', 'half', '-', 'crown', '##ds', 'for', 'me', '.', 'oh', ',', 'i', "'", 'll', 'go', 'carry', 'them', 'to', 'market', ',', 'mother', ',', 'and', 'nothing', 'but', 'vine', 'vol', '##k', 'shall', 'i', 'zee', ';', 'and', 'with', 'my', 'money', 'then', 'i', 'will', 'buy', 'land', ',', 'z', '##o', 'as', 'a', 'landlord', 'i', 'med', 'be', '.', "'", 'oh', 'my', 'dear', 'z', '##on', ',', 'wil', '##t', 'thee', 'know', 'me', ','

In [ ]:
print('Token to ids ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[0])))

Token to ids  [2821, 2388, 1010, 2026, 12136, 4328, 13687, 1045, 2097, 5271, 1010, 1998, 2035, 2005, 1037, 10647, 2004, 2017, 19960, 23727, 1025, 1998, 2007, 2026, 10647, 2059, 1045, 2097, 4965, 6763, 1010, 29536, 2099, 1045, 4618, 2031, 2698, 2005, 2026, 9502, 3240, 1012, 2821, 2388, 1010, 1045, 1005, 2222, 2275, 2068, 2035, 2104, 2256, 21863, 1010, 1998, 2698, 10338, 21868, 2453, 3382, 2005, 2000, 2022, 1025, 2021, 2698, 10338, 21868, 2030, 2698, 6178, 21863, 2015, 1010, 2045, 1005, 2222, 2022, 2698, 2431, 1011, 4410, 5104, 2005, 2033, 1012, 2821, 1010, 1045, 1005, 2222, 2175, 4287, 2068, 2000, 3006, 1010, 2388, 1010, 1998, 2498, 2021, 15351, 5285, 2243, 4618, 1045, 23727, 1025, 1998, 2007, 2026, 2769, 2059, 1045, 2097, 4965, 2455, 1010, 1062, 2080, 2004, 1037, 18196, 1045, 19960, 2022, 1012, 1005, 2821, 2026, 6203, 1062, 2239, 1010, 19863, 2102, 14992, 2113, 2033, 1010, 2043, 14992, 2038, 2102, 5407, 2307, 3573, 1997, 7177, 1029, 1005, 1005, 2821, 1010, 2026, 6203, 2388, 1010, 2129,

In [ ]:
#encode/tokenize each text in the list
max_len = 0
for t in text:
  #update maximum sentence length
  encoded_sent = tokenizer.encode(t, add_special_tokens=True)
  max_len = max(max_len, len(encoded_sent))

print('max sentence length: ', max_len)

Token indices sequence length is longer than the specified maximum sequence length for this model (1733 > 512). Running this sequence through the model will result in indexing errors


max sentence length:  14006


In [ ]:
def mask_inputs_for_bert(text, max_len):
  #tokenize all sentences and map the tokens to word ids
  input_ids = []
  attention_masks = []
  i = 0
  for t in text:
    if(i<3):
      print('Text', t)
    encoded_dict = tokenizer.encode_plus(
                      t,
                      add_special_tokens = True,
                      max_length = 512,
                      pad_to_max_length = True,
                      return_attention_mask = True,
                )
    if(i<3):
      print('dict', encoded_dict['input_ids'])
    input_ids.append(encoded_dict['input_ids'])

    attention_masks.append(encoded_dict['attention_mask'])

    i = i+1
  #convert to tensor and return
  input_ids = tf.convert_to_tensor(input_ids)
  attention_masks = tf.convert_to_tensor(attention_masks)
  return input_ids, attention_masks

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

train_label = le.fit_transform(train_label)
val_label = le.transform(val_label)

In [ ]:
train_inp, train_mask = mask_inputs_for_bert(train_input, max_len)
val_inp, val_mask = mask_inputs_for_bert(val_input, max_len)
train_label = tf.convert_to_tensor(train_label)
val_label = tf.convert_to_tensor(val_label)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Text Once upon a time there was a clever, wily peasant, whose tricks could be much talked about. The best story, however, is how he once got the best of the devil and made a fool of him. One day the peasant had been working in his field, and just as it was getting dark he was getting ready to go home when in the middle of his field he saw a pile of burning coals. Filled with amazement he walked toward it, and sitting on the top of the glowing coals there was a little black devil. 'You must be sitting on a treasure,' said the peasant. 'Yes indeed,' replied the devil, 'on a treasure that contains more gold and silver than you have ever seen in your life.' 'The treasure is in my field and belongs to me,' said the peasant. 'It is yours,' answered the devil, 'if for two years you will give me one half of everything your field produces. I have enough money, but I have a desire for the fruits of the earth.' The peasant entered into the bargain, saying, 'To prevent any dispute from arising abo

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Text 'You,' she said, 'come with me.' She led him to a hollow tree, a thick one. 'Climb this tree,' she said. 'Higher up there is a hole. Climb down into it. At the bottom you'll find a room where a dog is sitting on a chest. You must pick him up and set him on the floor. You can take as much money as you want from the chest, but then set the dog back where you found him. On the table there is a tinderbox. Put it in your pocket as well.' He did what she had told him to do and climbed down into the hole. Entering the room, he saw the dog seated on the chest. Its eyes were as large as a fist. 'See here,' said the soldier, 'you are a good fellow. You won't do anything to me!' He lifted the dog into his arms then set him on the floor. He opened the chest, which was filled with copper coins, and he filled his pockets with them. Then he closed the chest and set the dog back on it. He went into the second room, where there also was a light burning. He again saw a dog sitting on a chest, as in

In [ ]:
print('Train_input_shape', train_inp.shape)
print('Train_mask_shape', train_mask.shape)
print('Validation_input_shape', val_inp.shape)
print('Validation_mask_shape', val_mask.shape)
print('Train_label_shape', train_label.shape)
print('Validation_label_shape', val_label.shape)

Train_input_shape (684, 512)
Train_mask_shape (684, 512)
Validation_input_shape (229, 512)
Validation_mask_shape (229, 512)
Train_label_shape (684,)
Validation_label_shape (229,)


In [ ]:
#bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 182)
bert_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = 182)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
log_dir = 'gdrive/MyDrive'

In [ ]:
model_save_path = 'gdrive/MyDrive'

In [ ]:
callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath = model_save_path,\
                                                save_weights_only = True,\
                                                monitor = 'val_loss',\
                                                mode = 'min',\
                                                save_best_only = True),\
             tf.keras.callbacks.TensorBoard(log_dir = log_dir)]

print('\nBert Model ', bert_model.summary())

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate = 2e-5, epsilon = 1e-08)

bert_model.compile(loss = loss, optimizer = optimizer, metrics = [metric])

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  139958    
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 67,093,430
Trainable params: 67,093,430
Non-trainable params: 0
_________________________________________________________________

Bert Model  None


In [ ]:
print(train_label.numpy())


[ 0  7 22 45 51 45 31 46 56 44  2 35  3 24 52 35 28 15 52 23 44 40 40 23
 11 18 45 38 41 34 24 35 17 34 15  1 60 26  2 54 47 41 21 22 40  9 49 37
 26 52 43  3 13 13 47 21 19 36 25 22 49 10 11  0 11 10  6 25 59 18 39  8
 51 59 13 25 26 47 26 59  4 46 36  5 59  7 44 39 59 36 44 12 56 56 58 43
  0 41 23 54 26 58 33 11 16 13 21  3  2  7 47 59 47 30 54 10 41 44 15 27
 45 39 46 59  3 12 58 11 45 49 24 38 41 56 34 40 15 42 30 44 58 19  1 55
 41 53 39 56  9 31 35 31 41  3 17 42  9 11  2 35  1 37 26 58 29 51 24 51
 38 17  7 28 33 13 38  6 18 22 40 51  8 46 57 22 36 31 49  8 36 41 41 22
  9 46 44 34  5 35 29 59 20 22 39  0 41 22 51  1 45 26 42 28 27 33 30 35
 26  9 34 60 16 16 28 14 38 15 24 19 19 13 39 35 44 22 51 46 37 46 40 31
 16  4  0 36 13 36 55 22 54  2 22 41 40 12 31 40 29 29 19 58 22 48 14 21
 21 53  5  9 47 59 50 17 54 59 14 52 36 41  3 53  1 20 39 30 54 11 28 36
 32 52 50 13 39 55 53 19 51 27 51  9 41 25 33 51 57 44 20 54 38 17 51 39
 20 42 43 30 59 57 43 60 55 31 43 60 23 30 40 10 27

In [ ]:
history = bert_model.fit([train_inp, train_mask],\
                         train_label,\
                         batch_size = 16,\
                         epochs = 4,\
                         validation_data = ([val_inp, val_mask], val_label),\
                         callbacks = callbacks)

Epoch 1/4
43/43 [==============================] - 64s 1s/step - loss: 5.0993 - accuracy: 0.0175 - val_loss: 4.9379 - val_accuracy: 0.0218
Epoch 2/4
43/43 [==============================] - 48s 1s/step - loss: 4.7880 - accuracy: 0.0497 - val_loss: 4.6711 - val_accuracy: 0.0611
Epoch 3/4
43/43 [==============================] - 49s 1s/step - loss: 4.4708 - accuracy: 0.0921 - val_loss: 4.4093 - val_accuracy: 0.0873
Epoch 4/4
43/43 [==============================] - 50s 1s/step - loss: 4.1208 - accuracy: 0.1520 - val_loss: 4.1580 - val_accuracy: 0.1266


In [ ]:
#make predictions
bert_model_pred_probs = bert_model.predict(val_label)

8/8 [==============================] - 4s 19ms/step


In [ ]:
logits = bert_model_pred_probs.logits
bert_model_preds = tf.squeeze(tf.round(logits))
bert_model_preds

<tf.Tensor: shape=(8, 182), dtype=float32, numpy=
array([[ 0.,  0.,  0., ...,  0., -1., -1.],
       [ 0.,  0.,  0., ...,  0., -1., -1.],
       [ 0.,  0.,  0., ...,  0., -1., -1.],
       ...,
       [ 0.,  0.,  0., ...,  0., -1., -1.],
       [ 0.,  0.,  0., ...,  0., -1., -1.],
       [ 0.,  0.,  0., ...,  0., -1., -1.]], dtype=float32)>

In [ ]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [ ]:
bert_model_results = calculate_results(y_true = val_label,
                      y_pred = bert_model_preds)

ValueError: ignored

In [ ]:
print(val_inp.shape)
print(bert_model_preds.shape)


(229, 512)
(8, 182)
